In [52]:
#import dependencies
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import requests
import json
from pprint import pprint
from datetime import datetime

In [53]:
#assign usda api key
load_dotenv()
usda_api_key = os.getenv('usda_api_key')

In [81]:
#assign paramenter variables
# commodity_dict = {'CORN' : ['PRODUCTION', 'STOCKS'], 'SOYBEANS' : ['PRODUCTION', 'STOCKS']}

commodity = 'CORN'
statistic = 'PRODUCTION'
state = 'NE'
year = 2019
data_level = 'STATE'
data_columns = ['commodity_desc', 'statisticcat_desc', 'util_practice_desc', 'unit_desc', 'state_alpha', 'year', 'freq_desc', 'Value']

In [83]:
def pull_usda_data(usda_api_key, commodity, statistic, data_level, state, year, data_columns):

    #assign usda api url and establish which parameter will be fed as variables
    usda_url = f'http://quickstats.nass.usda.gov/api/api_GET/?key={usda_api_key}&source_desc=SURVEY&commodity_desc={commodity}&statisticcat_desc={statistic}&agg_level_desc={data_level}&state_alpha={state}&year_GE={year}&format=JSON'

    #get usda data via api and return as json
    usda_data = requests.get(usda_url).json()

    #put dictionary from returned json data into df
    usda_df = pd.DataFrame(usda_data['data'])

    #slice usda_df with assigned columns prior to cleaning data
    usda_df_clean = usda_df[data_columns]

    #clean Value column so datatype is numeric
    usda_df_clean['Value'] = usda_df_clean['Value'].apply(lambda x: x.replace(',', ''))
    usda_df_clean['Value'] = usda_df_clean['Value'].apply(lambda x: int(x))

    return usda_df_clean

In [85]:
test_df = pull_usda_data(usda_api_key, commodity, statistic, data_level, state, year, data_columns)
test_df.head()

,commodity_desc,statisticcat_desc,util_practice_desc,unit_desc,state_alpha,year,freq_desc,Value
0,CORN,PRODUCTION,GRAIN,$,NE,2019,ANNUAL,6784596000
1,CORN,PRODUCTION,GRAIN,$,NE,2018,ANNUAL,6392448000
2,CORN,PRODUCTION,GRAIN,$,NE,2017,ANNUAL,5639055000
3,CORN,PRODUCTION,GRAIN,$,NE,2016,ANNUAL,5643668000
4,CORN,PRODUCTION,GRAIN,$,NE,2015,ANNUAL,6043118000
